In [1]:
import numpy as np
import pandas as pd
import os
import sys
from scipy.optimize import minimize

import re
import ast

from abc_rc import RouteChoiceModel
from definition import Network, LinkTransition

## Model Definition

TODO
- Implement additional route choice models (e.g., Recursive Logit)
- Add three methods to the model class (calculate_transition_probability, get_param_size, is_valid)

In [2]:
# Import Example Implementation of RL from code/model.py
# you can also define your own model by extending the RouteChoiceModel class
from model import RL

## Estimation

TODO
- Change path to link, node, transition files and output directory
- Change model definition

In [3]:
LINK_FILE = "../data/input/matsuyama/ped/link.csv"
NODE_FILE = "../data/input/matsuyama/ped/node.csv"
TRANSITION_FILE = "../data/input/matsuyama/ped/transition.csv"
OUTPUT_DIR = "../data/output/matsuyama/ped"

# Read data
df_link = pd.read_csv(LINK_FILE)
df_node = pd.read_csv(NODE_FILE)
df_transition = pd.read_csv(TRANSITION_FILE)

network = Network(df_node, df_link)


In [4]:
# Create model
model = RL(network)

# Create transition list (not need to modify)
transition_list = [LinkTransition.from_dict(row, network, model) for row in df_transition.to_dict(orient="records")]
transition_list = [t for t in transition_list if t is not None]# remove None values

In [5]:
# Define functions for estimation
def compute_minus_ll(params: np.ndarray) -> float:
    ll = 0.0
    for transition in transition_list:
        ll += transition.calculate_log_likelihood(params)
    return -ll

def compute_hessian(params: np.ndarray) -> np.ndarray:
    h = 10 ** -4  # 数値微分用の微小量
    n = len(params)
    res = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            e_i, e_j = np.zeros(n), np.zeros(n)
            e_i[i] = 1
            e_j[j] = 1

            res[i][j] = (-compute_minus_ll(params + h * e_i + h * e_j)
                        + compute_minus_ll(params + h * e_i - h * e_j)
                        + compute_minus_ll(params - h * e_i + h * e_j)
                        - compute_minus_ll(params - h * e_i - h * e_j)) / (4 * h * h)
    return res

def tval(x: np.ndarray) -> np.ndarray:
    return x / np.sqrt(-np.diag(np.linalg.inv(compute_hessian(x))))

In [ ]:
# Perform Estimation
x0 = np.zeros(model.get_param_size())
res = minimize(compute_minus_ll, x0, method="Nelder-Mead")

t_val = tval(res.x)
LL0 = -compute_minus_ll(x0)
LL = -compute_minus_ll(res.x)
rho2 = 1 - LL / LL0
adj_rho2 = 1 - (LL - len(res.x)) / LL0
aic = -2 * LL + 2 * len(res.x)

result_str = f"""
sample number = {len(transition_list)}
    variables = [{', '.join(map(str, network.f_name))}]
    parameter = [{', '.join(map(str, res.x))}]
        t value = [{', '.join(map(str, t_val))}]
            L0 = {LL0}
            LL = {LL}
            rho2 = {rho2}
adjusted rho2 = {adj_rho2}
            AIC = {aic}
        discount = {model.get_beta(res.x) if hasattr(model, 'get_beta') else 'N/A'}
"""
print(result_str)

with open(os.path.join(OUTPUT_DIR, "result.txt"), "w") as f:
    f.write(result_str)


## Simulation

TODO
- Change path to link, node, transition files, parameter file and output directory
- Change model definition

In [6]:
LINK_FILE = "../data/input/matsuyama/ped/link.csv"
NODE_FILE = "../data/input/matsuyama/ped/node.csv"
TRANSITION_FILE = "../data/input/matsuyama/ped/transition.csv"  # not need to contain "NextLinkID" column
PARAMETER_FILE = "../data/output/matsuyama/ped/result.txt"
OUTPUT_DIR = "../data/output/matsuyama/ped/"


# Read data
df_link = pd.read_csv(LINK_FILE)
df_node = pd.read_csv(NODE_FILE)
df_transition = pd.read_csv(TRANSITION_FILE)

network = Network(df_node, df_link)

In [7]:
# Create model
model = RL(network)

# Create transition list (not need to modify)
transition_list = [LinkTransition.from_dict(row, network, model) for row in df_transition.to_dict(orient="records")]

In [8]:
# Read Parameter
with open(PARAMETER_FILE, encoding="utf-8") as f:
    text = f.read()

match = re.search(r"parameter\s*=\s*(\[[^\]]+\])", text)
if match:
    param_list = ast.literal_eval(match.group(1))  # list[float]
else:
    raise ValueError("Failed to extract parameters from input/result.txt")
params = np.array(param_list, dtype=np.float32)

In [9]:
# Perform Simulation
next_link_ids = []
for transition in transition_list:
    if transition is not None:
        next_link_id = model.choose_transition(transition, params)
        next_link_ids.append(next_link_id)
    else:
        next_link_ids.append(None)

df_transition["NextLinkID"] = next_link_ids
df_transition.to_csv(os.path.join(OUTPUT_DIR, "transition_simulated.csv"), index=False)

print(df_transition.head())

   TripID  LinkID  NextLinkID  DestinationNodeID
0       1    1026       179.0                599
1       1    2877      1026.0                599
2       1    1026       179.0                599
3       1     179      2535.0                599
4       1     178      2365.0                599
